 Les fonctions basiques :

In [7]:
#importation de la bibliotheque dont on a besoin dans notre programme
from copy import deepcopy
import time

t = [[1,2,3],[8,6,0],[7,5,4]]  #l'état de départ
tf =[[1,2,3],[8,0,4],[7,6,5]]  #l'état but

def estEtatFinal (t,tf):
  return t==tf

def position_case_vide(t): 
  # fonction dont son retour est une variable qui nous renseigne sur la position de la case vide dans le taquin
    for row in range(len(t)): 
        for col in range(len(t[row])):  
            if t[row][col] == 0 : 
                return (row,col)

def numero_dans_Case(t,x,y):
  """Cette fonction retourne le contenu de la case dont les coordonnées x,y sont données"""
  return t[x][y]

def permuter(t, pos, move):
  # fonction qui permute les cases et retour le nouveau taquin
    temp = deepcopy(t)
    i, j = pos  # ancienne position de vide
    x, y = move  # nouvelle position de vide
    temp[i][j], temp[x][y] = temp[x][y], temp[i][j]  # permutation
    return temp

def afficher(t):
  # """
  # affiche le taquin
  # """
  print()
  for i in range(len(t)):       #   len( [ [1,2,3] , [8,6,0] , [7,5,4] ] ) = 3
    for j in range(len(t[i])):  #   len( [1,2,3] ) = 3  
      print('+-----',end='')    
    print('+')
    for j in range(len(t[i])):
      if t[i][j] != 0:
        print('| ',t[i][j],' ',end='')
      else:
        print('|     ',end='')
    print('|')
  for j in range(len(t[i])):
    print('+-----',end='')
  print('+')

#initialisation d'une fonction qui teste la validité des coordonnées
def valid(x,y):
    return x>-1 and x<3 and y>-1 and y<3

#initialisation d'une fonction de transition de la case vide et retourne le nouveau taquin
def transitions(t) :
  """Cette fonction retourne une liste contenant les permutations possible de l'état donné de la liste t"""
  pos=position_case_vide(t)
  liste = []
  x = [1,0,-1,0]
  y = [0,1,0,-1]
  for i in range(4):
    if(valid(pos[0]+x[i],pos[1]+y[i])):
       liste.append([pos[0]+x[i],pos[1]+y[i]])
  derive=[]
  for i in liste :
    derive.append(permuter(t,pos,i))
  return derive

def colander(l,y):
  # Cette fonction permet de supprimer de la liste l les éléments qui sont déja existants dans la liste y.
  # On utilisera cette fonction avec les listes transtionsPossibles (y) et freeNodes (l) 
  # et avectranstionsPossibles (y) et closedNodes (l)
    l1=deepcopy(l)
    for x in l1:
        if x in y:
            del l[l.index(x)]

def colanderV2(l,y):
  # Cette fonction permet de supprimer de la liste l les éléments qui n'existent pas dans la liste y.
  #  On utilisera cette fonction dans le Backtracking (retour en arrière).
    l1=deepcopy(l)
    for x in l1:
        if not x in y: 
            del l[l.index(x)]

def backtracking(closedNodes):
  # """
  # retour en arrière pour suivre la route optimale du taquin initialau final
  # """
    global tf
    print("Starting the search for the route to the solution\n")
    start_time = time.time()
    route=[]
    now = deepcopy(tf)
    while True:
        route.append(now)
        min = closedNodes.index(now)
        transitionsPossibles=transitions(now)
        colanderV2(transitionsPossibles,closedNodes)
        if t in transitionsPossibles:
            route.append(transitionsPossibles[transitionsPossibles.index(t)])
            break
        for i in transitionsPossibles:
            if closedNodes.index(i) < min:
                min = closedNodes.index(i)
        now=closedNodes[min]     
    print("There is a solution, it takes {} steps".format(len(route)))
    for r in route[::-1]:
        print("\nStep : ",len(route)-route.index(r))
        afficher(r)
    print("The search for the route to the solution took %5.4f seconds \n" %(time.time() - start_time))

Recherche A Etoile :

In [9]:
def h(t,tf):
  # """
  # la fonction heuristique « h » qui retourne le nombre des cases mal placés sans tenir compte du case vide.
  # """
  nb=0
  for i in range(len(t)):
    for j in range(len(t[i])):
      if (t[i][j]!=tf[i][j])and (t[i][j]!=0):
        nb=nb+1
  return nb

def aetoile(t,tf):
  success=False#                               indique si on a atteint l’état but ou non.
  start = time.time() #                        temps d’exécution
  niveaux = 0 #                                Niveaux indique le niveau de cet état (le nombre de transitions faites à partir de l’état initial
  free_nodes = [] #                            free_nodes contient les nœuds à visiter
  free_nodes.append(t)
  closed_nodes = [] #                          Closed_nodes contiendra les éléments déjà visités
  while (free_nodes!=[]) and (not success) and (niveaux < 100): 
    # et on n’a pas atteint l’état but
    first_node = free_nodes[0] #              first_node contiendra le 1er élément de free_nodes
    print()
    print ('** pas', niveaux,': **')
    niveaux += 1
    afficher(first_node)
    free_nodes.remove(first_node) #           l’élémine de cette liste et l’ajoute à closed_nodes
    closed_nodes.append(first_node)
    generated_states = transitions(first_node)#generated_states prend les états possibles à générer à partir de l’état actuel jusqu’à l’état but

    for s in generated_states:#               On parcourt cette nouvelle liste
      if s == tf:#                            si l’un est l’état final, success devient true
        success = True
        print (' \n** pas', niveaux,': **')
        afficher(s)
        goal_node = s #                       goal_node recoit l’état actuel 
    free_nodes = free_nodes + generated_states# on ajoute les generated_states à free_nodes

#et on les ordonne selon leur somme f(x) de : niveau actuel qui représente g(x) et le nombre de cases mal placés qui représente h(x)
    free_nodes.sort(key = lambda el:(niveaux+h(el,tf))) #list.sort() a un paramètre key afin de spécifier une fonction qui peut être appelée sur chaque élément de la liste avant d'effectuer des comparaisons

  if niveaux == 100:
    print("Recherche non conclussive")
  else:
    print(" \n --> Goal trouvé aprés",niveaux," iterations .")
  print ('Time spent: %0.2fs' % (time.time()-start))

print (h(t,tf),"cases mal placées depuis l'etat initial")
aetoile(t,tf)

3 cases mal placées depuis l'etat initial

** pas 0 : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |  6  |     |
+-----+-----+-----+
|  7  |  5  |  4  |
+-----+-----+-----+

** pas 1 : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |  6  |  4  |
+-----+-----+-----+
|  7  |  5  |     |
+-----+-----+-----+

** pas 2 : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |  6  |  4  |
+-----+-----+-----+
|  7  |     |  5  |
+-----+-----+-----+
 
** pas 3 : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |     |  4  |
+-----+-----+-----+
|  7  |  6  |  5  |
+-----+-----+-----+
 
 --> Goal trouvé aprés 3  iterations .
Time spent: 0.01s


 Recherche en profondeur d'abord : 

In [10]:
trace= [] #               initialisation de la liste des traces 
#                         Trace contient les états parcourus par le taquin de l’état initial à l’état final
visited= [] #             initialisation de la liste des noeuds visités
success= False #          Success initialisée à false donne true si l’état final est atteind

def dfs(t, tf):
  global start 
  start= time.time()
  global success
  if (success==False and t not in visited):
#                         Si l’état actuel n’est pas encore visité et on n’a pas atteint l’état but on ajoute cet état à trace
    trace.append(t)
    if estEtatFinal(t,tf):#Si c’est l’état final, success devient True
      success=True
    visited.append(t) #    On ajoute l’état actuel à visited
    tab=transitions(t) #   La variable tab contient les transitions possibles à partir de l’état actuel
    for i in tab:
#On parcourt chaque élément de tab, s’il n’est pas visité déjà et si on n’a pas encore atteint l’état final, on reprend la fonction dés le début avec les nouveaux paramètres : la 1ere transition possible et l’état final jusqu’à atteindre l’état but.    C’est le concept de la récursivité
      if i not in visited and success==False:
         dfs(i,tf)
#appel a la fonction de resolution par methode de profondeur
dfs(t,tf) #On appelle après la fonction dfs pour s’exécuter et on affiche tous les états parcourus dans trace et leur niveau.
niveaux=0
for i in trace :
  print()
  print ('** pas', niveaux,' : **')
  afficher(i)
  niveaux+=1
print("\n --> Goal trouvé aprés",niveaux-1," iterations .")
print ('Time spent: %0.2fs' % (time.time()-start))




** pas 0  : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |  6  |     |
+-----+-----+-----+
|  7  |  5  |  4  |
+-----+-----+-----+

** pas 1  : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |  6  |  4  |
+-----+-----+-----+
|  7  |  5  |     |
+-----+-----+-----+

** pas 2  : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |  6  |  4  |
+-----+-----+-----+
|  7  |     |  5  |
+-----+-----+-----+

** pas 3  : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |     |  4  |
+-----+-----+-----+
|  7  |  6  |  5  |
+-----+-----+-----+

 --> Goal trouvé aprés 3  iterations .
Time spent: 0.01s


Recherche en largeur d'abord :

In [11]:
def bfs(t, tf):
  global start
  start=time.time()#         Start: récupère le temps de lancement de l’exécution
  visited=[] #                Visited: une liste qui contiendra les états visités 
  queue=[] #                  Queue: une liste qui contiendra les états à parcourir 

  queue.append(t) #           On ajoute l’état initial à queue
  visited.append(t) #         On ajoute l’état initial à visited
  trace=[] #                  Trace une liste qui contiendra les états parcourus partant de l’état initial à l’état final (Elles sont initialisées à vide) 

  while queue: #              queue n’est pas vide
    f=queue.pop(0) #          f reçoit le 1er élément de queue et l’élimine de cette liste
       
    if estEtatFinal(f,tf):#   On teste si f coïncide avec l’état final ou non. 
      trace.append(f)#        on ajoute f à trace 
      success=True#           et success devient true
      break 
    trace.append(f) #         Sinon on ajoute f à trace
    trs=transitions(f) #      trs reçoit les transitions possibles de f
    for j in trs: #           On parcourt ces transitions l’une après l’autre jusqu’à queue devient vide
     if j not in visited: #   s’elles ne sont pas dans visited
      visited.append(j)#      on les ajoute dans queue et dans visited
      queue.append(j)
  return trace


niveaux=0
trace=bfs(t, tf)
for i in trace : 
   print()
   print ('** pas', niveaux,': **')
   afficher(i)
   niveaux+=1
print("\n --> Goal trouvé aprés",niveaux-1," iterations . \n")
print ('Time spent: %0.2fs' % (time.time()-start))




** pas 0 : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |  6  |     |
+-----+-----+-----+
|  7  |  5  |  4  |
+-----+-----+-----+

** pas 1 : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |  6  |  4  |
+-----+-----+-----+
|  7  |  5  |     |
+-----+-----+-----+

** pas 2 : **

+-----+-----+-----+
|  1  |  2  |     |
+-----+-----+-----+
|  8  |  6  |  3  |
+-----+-----+-----+
|  7  |  5  |  4  |
+-----+-----+-----+

** pas 3 : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |     |  6  |
+-----+-----+-----+
|  7  |  5  |  4  |
+-----+-----+-----+

** pas 4 : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |  6  |  4  |
+-----+-----+-----+
|  7  |     |  5  |
+-----+-----+-----+

** pas 5 : **

+-----+-----+-----+
|  1  |     |  2  |
+-----+-----+-----+
|  8  |  6  |  3  |
+-----+-----+-----+
|  7  |  5  |  4  |
+-----+-----+-----+

** pas 6 : **

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-

Recherche en Profondeur limitee:

In [12]:
def recherche(t,depthFirst,star,limit):
    global end

    #On récupère la valeur du temps du système dans start_time
    start_time = time.time()

    #On initialise la liste freeNodes avec la valeur courante du taquin
    freeNodes=[t]
    
    #Si la valeur de limit est différente de -1 alors on initialise notre liste de niveau
    if limit != -1:
        level=[0]
    
    #On initialise la liste closedNodes à l'état vide
    closedNodes=[]

    #On affiche le taquin 
    print("L'état Initial")
    afficher(freeNodes[0])

    #On initialise la variable check (du succès) à False
    check = False

    #On initialise le nombre d'itérations à 0
    iterations = 0
    while len(freeNodes) != 0 :
      iterations += 1
      now = freeNodes.pop(0)
      closedNodes.append(now)
      if limit != -1 :
          currentlevel=level.pop(0)
          if currentlevel >= limit :
                  if estEtatFinal(now,tf):
                      check = True
                      print("\nEtat Final Atteint !")
                      afficher(now)
                      closedNodes.append(now)
                  if check:
                      break
                  continue
      transtionsPossibles=transitions(now)
      colander(transtionsPossibles,closedNodes)
      colander(transtionsPossibles,freeNodes)
      for i in transtionsPossibles:
          if estEtatFinal(i,tf):
                check = True
                print("\nEtat Final Atteint !")
                afficher(i)
                closedNodes.append(i)
                break
      if check:
            break
      if depthFirst:
            freeNodes = transtionsPossibles + freeNodes
            if limit != -1:
                level=[currentlevel+1]*len(transtionsPossibles) + level
      else:
            freeNodes = freeNodes + transtionsPossibles
            if limit != -1:
                level=level + [currentlevel+1]*len(transtionsPossibles)
      if star:
            freeNodes.sort(key=lambda e:(iterations + h(e)))    
      
    if check:    
        print("\nThe search for the solution took %5.4f seconds \n" %(time.time() - start_time) )
        backtracking(closedNodes)
    else:
        print("The solution haven't been reached so there is no path to it, the limit is",limit)
x=int(input("Please set the limit: "))
recherche(t,True,False,x)

Please set the limit: 22
L'état Initial

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |  6  |     |
+-----+-----+-----+
|  7  |  5  |  4  |
+-----+-----+-----+

Etat Final Atteint !

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |     |  4  |
+-----+-----+-----+
|  7  |  6  |  5  |
+-----+-----+-----+

The search for the solution took 0.0112 seconds 

Starting the search for the route to the solution

There is a solution, it takes 4 steps

Step :  1

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |  6  |     |
+-----+-----+-----+
|  7  |  5  |  4  |
+-----+-----+-----+

Step :  2

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |  6  |  4  |
+-----+-----+-----+
|  7  |  5  |     |
+-----+-----+-----+

Step :  3

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-----+-----+
|  8  |  6  |  4  |
+-----+-----+-----+
|  7  |     |  5  |
+-----+-----+-----+

Step :  4

+-----+-----+-----+
|  1  |  2  |  3  |
+-----+-